# Generating Summary including Sequential Information 

## LEAD3 as the baseline model - Using first 3 sentences of each article 

In [ ]:
#Return predicted summary
df_doc = test_df.text_clean
pred_summaries = [np.array(df_doc.iloc[j])[:3] for j in range(len(df_doc))]
pred_summaries = [summ_list if type(summ_list) == str else ' '.join(summ_list) for summ_list in pred_summaries]

#Match with gold summaries
df_gold = test_df.summary_clean
gold_summaries = [' '.join(df_gold .iloc[j]) for j in range(len(pred_summaries))]
summaries_comp = tuple(zip(pred_summaries, gold_summaries))
scores = rougeScores(pred_summaries, gold_summaries, keys=['rouge1', 'rougeL'], use_stemmer=True)
model_df = pd.DataFrame({'ModelID' : ['LEAD3:Rouge1', 'LEAD3:RougeL']})
res_LEAD3 = pd.DataFrame(columns=['recall' , 'precision', 'f1'])
res_LEAD3 = res_LEAD3.append(pd.DataFrame(scores).T, ignore_index=True)
res_LEAD3 = pd.concat([model_df, res_LEAD3],axis = 1)
res_LEAD3

,ModelID,recall,precision,f1
0,LEAD3:Rouge1,0.564235,0.754172,0.574051
1,LEAD3:RougeL,0.547237,0.728264,0.557038


In [ ]:
summaries_comp[0][0]

'By MATT SCHWARTZ in Houston and WENDELL JAMIESON in New York Daily News Writers\n\nSaturday, October 14th 1995, 4:22AM\n\nBleeding from a massive chest wound, Tejano star Selena cried, "Help me! Help me! I\'ve been shot!"'

In [ ]:
#Update sumarry dictionary and save to pickle file
summaryComp.update({'summaries_comp_LEAD3': summaries_comp})
# with open('Data/summaryComp.pickle', 'wb') as handle:                                     
#     pickle.dump(summaryComp, handle)

## Logistic Regression

In [ ]:
#Define Model
model = LogisticRegression(class_weight='balanced', random_state=42)
#Fit model
model.fit(xTrain,yTrain)
#Predict Model
y_pred = model.predict_proba(xTest)
    
#Convert to binary predictions
y_pred_bin = (y_pred >=0.5)*1

cm = confusion_matrix(yTest, y_pred_bin[:,1], labels=[0,1])
   
#Return predicted summaries
idx, doc_index, pred_summaries = predict_summaries(Test_doc_label,  y_pred[:,0], test_df.text_clean)

#Match with gold summaries
df_gold = test_df.summary_clean[doc_index]
gold_summaries = [' '.join(df_gold .iloc[j]) for j in range(len(pred_summaries))]
summaries_comp = tuple(zip(pred_summaries, gold_summaries))

scores = rougeScores(pred_summaries, gold_summaries,  keys=['rouge1', 'rougeL'], use_stemmer=True)
model_df = pd.DataFrame({'ModelID' : ['LR_bal:Rouge1', 'LR_bal:RougeL']})
res_LR_bal = pd.DataFrame(columns=['recall' , 'precision', 'f1'])
res_LR_bal = res_LR_bal.append(pd.DataFrame(scores).T, ignore_index=True)
res_LR_bal = pd.concat([model_df, res_LR_bal],axis = 1)
res_LR_bal

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,ModelID,recall,precision,f1
0,LR_bal:Rouge1,0.225872,0.244652,0.203367
1,LR_bal:RougeL,0.144019,0.164116,0.132021


In [ ]:
summaries_comp[0][0]

'He felt for a pulse in her neck but could feel only twitching muscles, he said. "It was covered with blood. Prosecutors say it was deliberate.'

In [ ]:
#Update sumarry dictionary and save to pickle file
summaryComp.update({'summaries_comp_LR_bal': summaries_comp})
# with open('Data/summaryComp.pickle', 'wb') as handle:                                     
#     pickle.dump(summaryComp, handle)

## LSTM (Long Short Term Memory Model)

**Prepare the dataset for taining the LSTM**

In [ ]:
fn = lambda x: np.array(x)
tmp_df = train_df.copy()
tmp_df.text_embedding = tmp_df.text_embedding.apply(fn)
tmp_df.labels= tmp_df.labels.apply(fn)
tmp_df.text_embedding = tmp_df.text_embedding.apply(lambda x: x.reshape(1, x.shape[0],x.shape[1]))
tmp_df.labels = tmp_df.labels.apply(lambda x: x.reshape(1, len(x),1))
l_xTrain = tmp_df.text_embedding.tolist()
l_yTrain = tmp_df.labels.tolist()

tmp_df = test_df
tmp_df.text_embedding = tmp_df.text_embedding.apply(fn)
tmp_df.labels= tmp_df.labels.apply(fn)
tmp_df.text_embedding = tmp_df.text_embedding.apply(lambda x: x.reshape(1, x.shape[0],x.shape[1]))
tmp_df.labels = tmp_df.labels.apply(lambda x: x.reshape(1, len(x),1))
l_xTest = tmp_df.text_embedding.tolist()
l_yTest = tmp_df.labels.tolist()

In [ ]:
print('shape of training sample for first 3 sentences', l_xTrain[0].shape, l_xTrain[1].shape, l_xTrain[2].shape)
print('shape of label for first 3 sentences', l_yTrain[0].shape, l_yTrain[1].shape, l_yTrain[2].shape)
print('We see that each sentences has different length and they are labled accordingly')

shape of training sample for first 3 sentences (1, 32, 768) (1, 40, 768) (1, 19, 768)
shape of label for first 3 sentences (1, 32, 1) (1, 40, 1) (1, 19, 1)
We see that each sentences has different length and they are labled accordingly


In [ ]:
# define LSTM
model = Sequential([
                    # LSTM(32, input_shape=(None, 768), return_sequences=True, dropout=0),   # Uncomment to train LSTM with 32 nodes.
                    # LSTM(64, input_shape=(None, 768), return_sequences=True, dropout=0),   # Uncomment to train LSTM with 64 nodes.
                    # Bidirectional(LSTM(32, return_sequences=True, dropout=0), input_shape=(None, 768)), # Uncomment to train biLSTM with 32 nodes.
                    Bidirectional(LSTM(64, return_sequences=True, dropout=0), input_shape=(None, 768)),   # Uncomment to train biLSTM with 64 nodes.
                    TimeDistributed(Dense(1, activation='sigmoid'))
                    ])

metrics=[SensitivityAtSpecificity(0.5, num_thresholds=1), SpecificityAtSensitivity(0.5, num_thresholds=1)]
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=metrics)
# train LSTM
train_loss =[]
for j in range(len(l_xTrain)):   
    X, y = l_xTrain[j], l_yTrain[j]
    history = model.fit(X, y, epochs=1, batch_size=1, verbose=0)
    train_loss.append(history.history['loss'])

In [ ]:
## save training model
# model.save('LSTM_25.h5')  # creates a HDF5 file 'my_model.h5'
# model.save('LSTM_50.h5')  # creates a HDF5 file 'my_model.h5'
# model.save('LSTM_bi_25.h5')  # creates a HDF5 file 'my_model.h5'
model.save('LSTM_bi_50.h5')  # creates a HDF5 file 'my_model.h5'

In [ ]:
model_arc = ['LSTM_25', 'LSTM_50', 'LSTM_bi_25', 'LSTM_bi_50']

for m in model_arc:
    model = load_model(m + '.h5')
    print(model.summary())

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, None, 25)          79400     
                                                                 
 time_distributed_1 (TimeDis  (None, None, 1)          26        
 tributed)                                                       
                                                                 
Total params: 79,426
Trainable params: 79,426
Non-trainable params: 0
_________________________________________________________________
None
Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, None, 50)          163800    
                                                                 
 time_distributed_2 (TimeDis  (None, None, 1)          51        
 tributed)             

In [ ]:
res_LSTM = pd.DataFrame()
#Predict output for test data
model_arc = ['LSTM_25', 'LSTM_50', 'LSTM_bi_25', 'LSTM_bi_50']

for m in model_arc:
    model = load_model(m + '.h5')
    model_df = pd.DataFrame({'ModelID' : [m+':Rouge1', m+':RougeL']})

    # evaluate LSTM
    y_pred_list =[]
    idx_list=[]
    for j in range(len(l_xTest)):
        X= l_xTest[j]
        y_pred = model.predict(X, verbose=0)
        idx = np.argsort(y_pred[0].flatten())[-3:]
        idx = sorted(idx)
        y_pred_list.append(y_pred)
        idx_list.append(idx)
    pred_summaries = [' '.join(np.array(tmp_df.text_clean.iloc[j])[np.array(idx_list[j])].tolist()) 
                      for j in range(len(idx_list))]
    df_gold = tmp_df.summary_clean
    gold_summaries = [' '.join(df_gold .iloc[j]) for j in range(len(pred_summaries))]
    summaries_comp = tuple(zip(pred_summaries, gold_summaries))
    #calculate rouge score
    scores = rougeScores(pred_summaries, gold_summaries, keys=['rouge1', 'rougeL'], use_stemmer=True)
    summaryComp.update({'summaries_comp_'+m: summaries_comp})

    tmp = pd.DataFrame(columns=['recall' , 'precision', 'f1'])
    tmp = tmp.append(pd.DataFrame(scores).T, ignore_index=True)
    tmp = pd.concat([model_df, tmp],axis = 1)
    res_LSTM = pd.concat([res_LSTM, tmp], ignore_index=True)

In [ ]:
summaries_comp[0][0]

'By MATT SCHWARTZ in Houston and WENDELL JAMIESON in New York Daily News Writers\n\nSaturday, October 14th 1995, 4:22AM\n\nBleeding from a massive chest wound, Tejano star Selena cried, "Help me! Help me! Shaken witnesses yesterday told a spellbound Houston courtroom how the blood-covered, mortally wounded 23-year-old Hispanic singing sensation burst into the lobby of the Corpus Christi Days Inn last March 31.\n\n'

In [ ]:
res_LSTM

,ModelID,recall,precision,f1
0,LSTM_25:Rouge1,0.548410,0.761568,0.568592
1,LSTM_25:RougeL,0.530603,0.734615,0.550832
2,LSTM_50:Rouge1,0.554123,0.762394,0.573243
3,LSTM_50:RougeL,0.536463,0.735191,0.555563
4,LSTM_bi_25:Rouge1,0.552088,0.760683,0.570516
5,LSTM_bi_25:RougeL,0.533998,0.733990,0.552702
6,LSTM_bi_50:Rouge1,0.555638,0.764624,0.575063
7,LSTM_bi_50:RougeL,0.537943,0.737781,0.557474


In [ ]:
#Update sumarry dictionary and save to pickle file
with open('Data/summaryComp.pickle', 'wb') as handle:                                     
    pickle.dump(summaryComp, handle)

In [ ]:
# Merge all the results
allModel = [res_TR, res_LR_default, res_LR_elasticNet, res_NN, res_LEAD3, res_LR_bal, res_LSTM]
res_all = pd.concat(allModel, ignore_index=True)
res_all

,ModelID,recall,precision,f1
0,TR:Rouge1,0.272896,0.315467,0.250868
1,TR:RougeL,0.195855,0.233583,0.182307
2,LR_default:Rouge1,0.494643,0.708379,0.514864
3,LR_default:RougeL,0.469454,0.671882,0.489722
4,LR_elasticNet:Rouge1,0.508569,0.727348,0.532209
5,LR_elasticNet:RougeL,0.485610,0.693151,0.508884
6,NN_16:Rouge1,0.555342,0.746306,0.565705
7,NN_16:RougeL,0.537553,0.718952,0.547235
8,NN_32_8:Rouge1,0.560763,0.729521,0.563156
9,NN_32_8:RougeL,0.541522,0.700605,0.543300


In [ ]:
res_all.to_csv('Data/resultData.csv')

In [ ]:
# Do not run this cell untill all the models has been evaluated
# uncomment following to save new results if all the model has been retrained
# with open('Data/allResults.pickle', 'wb') as handle:                                     
#     pickle.dump(res_all, handle)